## Visualización de activaciones en una CNN entrenada con MNIST
Este Notebook  ilustra cómo una red neuronal convolucional (CNN) extrae características visuales jerárquicas, desde líneas simples hasta formas más complejas en capas profundas.

Usaremos el conjunto de datos MNIST y visualizaremos los mapas de activación (feature maps) generados por las primeras capas de la CNN.

## Elaborado por: Luis Eduardo Ordoñez

### Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Biblioteca principal de TensorFlow
import tensorflow as tf

# Clases y funciones necesarias para construir modelos de Keras
from tensorflow.keras import Input, Model, layers, models

# Capas específicas para construir una red neuronal convolucional
from tensorflow.keras.layers import Conv2D, Flatten, Dense

# Conjunto de datos MNIST desde Keras (imágenes de dígitos manuscritos)
from tensorflow.keras.datasets import mnist

### Cargar y preparar los datos

In [ ]:
# Carga el conjunto de datos MNIST, que contiene imágenes de dígitos manuscritos
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Selecciona solo una imagen del conjunto de prueba para ilustrar (la segunda imagen, índice 1)
x_test = x_test[1]

# Ajusta la forma del array para que sea compatible con el modelo: (1, 28, 28, 1)
# Se añade una dimensión para el lote (batch) y otra para el canal (imagen en escala de grises)
x_test = x_test.reshape((-1, 28, 28, 1)) / 255.0  # Además, normaliza los valores de píxeles entre 0 y 1

### Construir una CNN simple

In [ ]:
# Definir la entrada de la red: imágenes de 28x28 píxeles con 1 canal (escala de grises)
inputs = Input(shape=(28, 28, 1))

# Primera capa convolucional con 8 filtros de 3x3 y activación ReLU
x = Conv2D(8, (3, 3), activation='relu', name='conv1')(inputs)

# Segunda capa convolucional con 16 filtros de 3x3 y activación ReLU
x = Conv2D(16, (3, 3), activation='relu', name='conv2')(x)

# Aplanar la salida de la última convolución para conectar con capas densas
x = Flatten()(x)

# Capa densa de salida con 10 neuronas (una por clase) y activación softmax
outputs = Dense(10, activation='softmax')(x)

# Crear el modelo final especificando las entradas y salidas
model = Model(inputs=inputs, outputs=outputs)

# Compila el modelo especificando:
# - Optimizador: 'adam', que ajusta los pesos automáticamente durante el entrenamiento
# - Función de pérdida: 'sparse_categorical_crossentropy', adecuada para clasificación multiclase con etiquetas enteras
# - Métrica de evaluación: 'accuracy' para monitorear la precisión del modelo durante el entrenamiento
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Entrena el modelo usando el conjunto de entrenamiento
# - Las imágenes se normalizan dividiendo por 255.0
# - Se realiza el entrenamiento por 1 época (pasada completa por los datos)
# - Se usa un tamaño de lote (batch_size) de 128 ejemplos por iteración
model.fit(x_train.reshape(-1,28,28,1)/255.0, y_train, epochs=1, batch_size=128)

### Crear un modelo que devuelva activaciones

In [ ]:
# Obtiene las salidas (activaciones) de todas las capas convolucionales del modelo
layer_outputs = [layer.output for layer in model.layers if 'conv' in layer.name]

# Crea un nuevo modelo que, en lugar de predecir una clase, devuelve las activaciones intermedias
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

# Genera las activaciones para la imagen de prueba
activations = activation_model.predict(x_test)

# Obtiene los nombres de las capas convolucionales para usar como títulos en las gráficas
layer_names = [layer.name for layer in model.layers if 'conv' in layer.name]

# Itera por cada capa y sus activaciones correspondientes
for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1]  # Número de mapas de activación (filtros)
    size = layer_activation.shape[1]         # Altura y ancho de cada mapa

    # Crea una cuadrícula para visualizar todos los mapas de activación uno al lado del otro
    display_grid = np.zeros((size, size * n_features))

    for i in range(n_features):
        activation = layer_activation[0, :, :, i]  # Extrae un mapa de activación

        # Normaliza los valores del mapa para mejorar la visualización
        activation -= activation.mean()
        activation /= (activation.std() + 1e-5)
        activation *= 64
        activation += 128
        activation = np.clip(activation, 0, 255).astype('uint8')

        # Inserta el mapa normalizado en la cuadrícula
        display_grid[:, i * size: (i + 1) * size] = activation

    # Muestra la cuadrícula con las activaciones de una capa convolucional
    scale = 1.5
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(f'Activaciones de la capa: {layer_name}')
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    plt.show()

### ¿Qué muestra este ejemplo?
* Las primeras capas resaltan bordes y contornos simples.
* Las capas más profundas combinan esas formas en patrones más complejos.
* Ayuda a visualizar cómo la red "comprende" una imagen paso a paso.